In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
from scipy.stats import linregress
from matplotlib.colors import LightSource
from matplotlib import cm
import pickle
#from landlab import RasterModelGrid, imshow_grid
import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from scipy.signal import savgol_filter
from scipy.stats import linregress
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.gridspec import GridSpec

In [ ]:
sims = ["025m_1", "05m_2", "10m_3",]
colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00"]

lw = 2.5

r = 0
c = 0

fig, ax = plt.subplots(3, 3, figsize = (12, 9))
for sim in sims:
    #Pulls the data for the simulation
    f = "time-dependent-experiments_" + sim + ".csv"
    data = pd.read_csv(f, index_col = 0)
    data = data.T
    data = data.reset_index(inplace = False)
    data['days'] = data['index'].astype(float)
    data = data.iloc[1:]

    #Find the inversion time
    df = pd.read_csv("time-dependent-experiments_" + sim + "_s_trans.csv")

    for col in df.columns:
        if df[col].iloc[100] == df[col].max():
            inversion = float(col)
            print(inversion)
            break
    

    ax1b = ax[r][c].twinx()
    lns3 = ax[r][c].plot(data['days'], data['ice_height_m'], c = '#0072B2', label = "Ice Height ($s'$)", linewidth = lw)
    lns2 = ax[r][c].plot(data['days'], data['cone_height_m'], c = 'k', label = "Cone Height ($z'$)", linestyle = 'dashed', linewidth = lw)
    lns1 = ax1b.plot(data['days'], data['cone_height_m']-data['ice_height_m'], c = 'gray', linestyle = 'dashdot', label = "Debris Depth ($h'$)", zorder = 0, linewidth = lw)
    ax[r][c].set_ylabel("$z'$, $s'$ (m)", fontsize = 11)
    ax1b.set_ylabel("$h'$ (m)", fontsize = 11, rotation = 270, va = 'bottom')
    ax[r][c].set_ylim([-1.33, 1.33])
    ax1b.set_ylim([-0.03, 1.03])

    
    r += 1
    lns6 = ax[r][c].plot(data['days'], savgol_filter(data['ice_height_m'], window_length = 6, polyorder = 2, deriv = 1, delta = 0.5), c = '#56B4E9', label = "$\partial s'/\partial t$", linewidth = lw)
    lns5 = ax[r][c].plot(data['days'], savgol_filter(data['cone_height_m'], window_length = 6, polyorder = 2, deriv = 1, delta = 0.5), c = '#009E73', linestyle = 'dashed', label = "$\partial z'/\partial t$", linewidth = lw)
    lns4 = ax[r][c].plot(data['days'], savgol_filter(data['cone_height_m']-data['ice_height_m'], window_length = 6, polyorder = 2, deriv = 1, delta = 0.5), c = '#D55E00', linestyle = 'dashdot', label = "$\partial h'/\partial t$", linewidth = lw)
    ax[r][c].vlines(inversion, ymin = -0.022, ymax = 0.025, color = 'k', linewidth = 0.5)
    ax[r][c].set_ylabel("$\partial s'/ \partial t$, $\partial z'/ \partial t$, $\partial h'/\partial t$ (m/day)", fontsize = 11)
    #ax[r][c].annotate("Pit\ninversion", xy = (inversion, 0.029), xycoords = 'data', fontsize = 10, ha = 'center')
    ax[r][c].annotate("$b_0 = 0.04$ m/day\n$D=0.009$ m/day", xy = (0.57, 0.86), xycoords = "axes fraction", fontsize = 9)
    ax[r][c].set_ylim([-0.025, 0.045])
    ax[r][c].set_yticks([-0.02, -0.01, 0, 0.01, 0.02, 0.03, 0.04])
    
    
    r += 1
    lns7 = ax[r][c].plot(data['days'], savgol_filter(2 * data['cone_height_m']/data['cone_width_m'], window_length = 5, polyorder = 2, deriv = 0, delta = 0.5), label = "Mean Slope", color = '#E69F00', linestyle = 'solid', linewidth = lw)
    ax[r][c].vlines(inversion, ymin = -0.05, ymax = 0.65, color = 'k', linewidth = 0.5)
    ax[r][c].set_ylim([-0.05,0.65])
    ax[r][c].set_ylabel('Slope', fontsize = 11)
    ax[r][c].set_xlabel('Time (days)', fontsize = 11)
    ax[r][c].annotate("$S_c$ = 1.15", xy = (0.05, 0.93), xycoords = 'axes fraction', fontsize = 9)
    
    lns = lns1+lns2+lns3+lns4+lns5+lns6+lns7
    labs = [l.get_label() for l in lns]

    r = 0
    c += 1

ax[0][0].annotate("$d_{pit} = 0.25$ m", xy = (0.66, 0.45), xycoords = "axes fraction", fontsize = 9)
ax[0][0].annotate("$w_{pit} = 0.5$ m", xy = (0.66, 0.35), xycoords = "axes fraction", fontsize = 9)

ax[0][1].annotate("$d_{pit} = 0.5$ m", xy = (0.66, 0.45), xycoords = "axes fraction", fontsize = 9)
ax[0][1].annotate("$w_{pit} = 0.5$ m", xy = (0.66, 0.35), xycoords = "axes fraction", fontsize = 9)

ax[0][2].annotate("$d_{pit} = 1.0$ m", xy = (0.66, 0.45), xycoords = "axes fraction", fontsize = 9)
ax[0][2].annotate("$w_{pit} = 0.5$ m", xy = (0.66, 0.35), xycoords = "axes fraction", fontsize = 9)

ax[0][0].annotate("(a)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[0][1].annotate("(b)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[0][2].annotate("(c)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)

ax[1][0].annotate("(d)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[1][1].annotate("(e)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[1][2].annotate("(f)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)

ax[2][0].annotate("(g)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[2][1].annotate("(h)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)
ax[2][2].annotate("(i)", xy = (-0.15, 1.01), xycoords = "axes fraction", fontsize = 11)

labs = [l.get_label() for l in lns]
fig.legend(lns, labs, ncol = 7, bbox_to_anchor = (0.905, 1.03), handlelength = 2.8, fontsize = 9)

plt.tight_layout()
plt.savefig('time-dependent-behavior.png', dpi = 300, bbox_inches = 'tight')
plt.show()

In [ ]:
sims = ["025m_1", "05m_2", "10m_3",]
colors = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#D55E00"]

lw = 2.5
labels = ['0.25', '0.5', '1.0']
lns = []

fig, ax = plt.subplots(figsize = (4,3.5))
i = 0
for sim in sims:
    #Pulls the data for the simulation
    f = "time-dependent-experiments_" + sim + ".csv"
    data = pd.read_csv(f, index_col = 0)
    data = data.T
    data = data.reset_index(inplace = False)
    data['days'] = data['index'].astype(float)
    data = data.iloc[1:]
    
    #ax.plot(data['days'][3:], data['cone_height_m'][3:]/(data['days'][3:]*0.04), c = colors[i], label = labels[i], linewidth = 3)
    ax.plot(data['days'][1:]*0.04/(59.28*0.04), data['cone_height_m'][1:]/(data['days'][1:]*0.04), c = colors[i], label = labels[i], linewidth = 3)
    #ax.plot(np.linspace(0,2,10), 0.5 * np.linspace(0,2,10), c= 'k', linestyle = 'dashed')
    i += 1

ax.set_xlabel("Bare Ice Melt (m), $\\Delta s_0$", fontsize = 11)
ax.set_ylabel("$z'/\\Delta s_0$", fontsize = 11)
plt.legend(title = 'Pit Depth (m)')
sns.despine()
plt.savefig('percent-height.png', dpi = 300, bbox_inches = 'tight')
plt.show()